<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_0812_MutiLNEuler_G4_torch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Euler 實驗結果測試
<a href="https://drive.google.com/file/d/1RzHAFqRE0sV5uXkwbzjXXo0uOHL8SFRo/view?usp=sharing">請見paper</a><br>
把每一層都做LN<br>
要對 G4 做正規化

In [11]:
from google.colab import drive
drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [3]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Flatten, LayerNormalization, Input,Activation, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from random import choice, sample
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_mod(primeTable25, num):
  return [num%index for index in primeTable25]

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def compute_coprimes(n): ## 獲得質因數 zero padding 到40
    coprimes = []
    lenth = 50
    for num in range(1,n):
        if gcd(num , n) == 1:
            coprimes.append(num)
        if len(coprimes) >= lenth:
            return coprimes
    coprimes = coprimes + [0] * (lenth - len(coprimes))
    return coprimes

def G4(num):
  return num / (np.log(num/2))**2

def G4_layer(x):
  return x / tf.math.square(tf.math.log(x / 2.0))

def prime_factors(n):
      i = 2
      factors = []
      while i * i <= n:
          if n % i:
              i += 1
          else:
              n //= i
              factors.append(i)
      if n > 1:
          factors.append(n)
      return factors

def goldbach_approx(n):
    C2 = 0.66016  # 孪生素数常数
    product = 1
    for p in prime_factors(n):
        if p == 2:
            continue
        product *= (p - 1) / (p - 2)
    return 2 * C2 * n / (math.log(n))**2 * product


In [4]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_100.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [5]:
print (len(trainDataNumbers))

100


In [6]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [7]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [8]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:

    # Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    mergedTrainNumber.append(feature)

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " done.")

#print (mergedTrainNumber)

merge number  6145  done.
merge number  4374  done.
merge number  15973  done.
merge number  10695  done.
merge number  11262  done.
merge number  19280  done.
merge number  12894  done.
merge number  8081  done.
merge number  12454  done.
merge number  5161  done.
merge number  2462  done.
merge number  15075  done.
merge number  19585  done.
merge number  9390  done.
merge number  19272  done.
merge number  8683  done.
merge number  1103  done.
merge number  8933  done.
merge number  12555  done.
merge number  9222  done.
merge number  14436  done.
merge number  11140  done.
merge number  13176  done.
merge number  9206  done.
merge number  11039  done.
merge number  23145  done.
merge number  5291  done.
merge number  10458  done.
merge number  5079  done.
merge number  91  done.
merge number  7447  done.
merge number  23327  done.
merge number  8752  done.
merge number  9016  done.
merge number  12475  done.
merge number  9125  done.
merge number  9190  done.
merge number  5862  do

In [9]:
print (mergedTrainNumber[1])
print (mergedTrainLabel[1])

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99]
4374


In [10]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
print ("X Train [0] value : \n",X_train[0])
print ("X Train [0] label : ",y_train[0])
print ("=======")
print ("X shpae : " , X_train[0].shape)

X Train [0] value : 
 [ 1  3  5  7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47
 49 51 53 55 57 59 61 63 65 67 69 71 73 75 77 79 81 83 85 87 89 91 93 95
 97 99]
X Train [0] label :  5024
X shpae :  (50,)


In [13]:
print (X_train[0].shape)

(50,)


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader



def G4_layer(x):
    return x / torch.square(torch.log(x / 2.0))
    # return torch.log(x)

def inverse_G4_layer(x):
    # return x * torch.sqrt(torch.exp(x*2.0))
    # return torch.exp(x)
    # return torch.exp(torch.sqrt(x)) * 2
    return x

# create torch tensor data
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_val_tensor  = torch.from_numpy(X_val).float()
y_val_tensor  = torch.from_numpy(y_val).float()

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_val_tensor,  y_val_tensor)

# create data loader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

input_size = X_train_tensor.shape[1]

# define model
class LN_Layer(nn.Module):
    def __init__(self, in_features, u1, u2):
        super(LN_Layer, self).__init__()
        self.mask_weights = nn.Parameter(torch.randn(1)) # G4控制閥值
        self.dense1 = nn.Linear(in_features, u1)
        self.dense2 = nn.Linear(in_features, u2)



    def forward(self, x):
      # group1 = x
      # group2 = F.relu(x)
      # # group2 = G4_layer(group2 + 1)
      # group2 = torch.log(group2 + 1)

      # group1 = F.relu(self.dense1(group1))
      # # group2 = inverse_G4_layer(group2)
      # group2 = torch.exp(self.dense1(group2))


      # group2 = F.relu(x)

      # merged = torch.cat([group1, group2], dim=1)
      # return merged

      group1 = x
      group2 = F.relu(x)
      group2 = G4_layer(group2 + 1)

      group1 = F.relu(self.dense1(group1))
      group2 = inverse_G4_layer(self.dense2(group2))


      merged = torch.cat([group1, group2], dim=1)
      return merged


model = nn.Sequential(
  LN_Layer(input_size, 50, 200),
  LN_Layer(250, 120, 80),
  LN_Layer(200, 120, 80),
  nn.Linear(200, 1)
)

# define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)



num_epochs = 200
for epoch in range (num_epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data

    # Clear gradients before each pass
    optimizer.zero_grad()

    # Forward pass
    outputs = model(inputs)

    # Calculate loss
    loss = criterion(outputs, labels)

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

  # Print training progress (optional)
  running_loss += loss.item()
  # Print every 100 batches
  print(f"ephch : {epoch}\t loss : {running_loss}")
  running_loss = 0.0

torch.save(model,"0813_torch.pt")
torch.save(model.state_dict(), "0813_weights_torch.pt")

print ("model saved")




# def forward(self, x):
    #     mask = torch.sigmoid(self.mask_weights) > 0.95

    #     group1 = x
    #     #group1 = torch.where(mask, G4_layer(x), x)
    #     group2 = F.relu(x)
    #     group2 = torch.log(group2 + 1)

    #     group1 = F.relu(self.dense1(group1))
    #     group2 = torch.exp(self.dense2(group2))


    #     merged = torch.cat([group1, group2], dim=1)
    #     return merged

ephch : 0	 loss : nan
ephch : 1	 loss : nan
ephch : 2	 loss : nan
ephch : 3	 loss : nan
ephch : 4	 loss : nan
ephch : 5	 loss : nan
ephch : 6	 loss : nan
ephch : 7	 loss : nan
ephch : 8	 loss : nan
ephch : 9	 loss : nan
ephch : 10	 loss : nan
ephch : 11	 loss : nan
ephch : 12	 loss : nan
ephch : 13	 loss : nan
ephch : 14	 loss : nan
ephch : 15	 loss : nan
ephch : 16	 loss : nan
ephch : 17	 loss : nan
ephch : 18	 loss : nan
ephch : 19	 loss : nan
ephch : 20	 loss : nan
ephch : 21	 loss : nan
ephch : 22	 loss : nan
ephch : 23	 loss : nan
ephch : 24	 loss : nan
ephch : 25	 loss : nan
ephch : 26	 loss : nan
ephch : 27	 loss : nan
ephch : 28	 loss : nan
ephch : 29	 loss : nan
ephch : 30	 loss : nan
ephch : 31	 loss : nan
ephch : 32	 loss : nan
ephch : 33	 loss : nan
ephch : 34	 loss : nan
ephch : 35	 loss : nan
ephch : 36	 loss : nan
ephch : 37	 loss : nan
ephch : 38	 loss : nan
ephch : 39	 loss : nan
ephch : 40	 loss : nan
ephch : 41	 loss : nan
ephch : 42	 loss : nan
ephch : 43	 loss : na

In [24]:
print (X_train_tensor.shape)

torch.Size([80, 50])


# predict

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")


# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

---loading primes table---
[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
## load data
from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_0325_mltiEuler_4layer_2.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set_G4_7M.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
testDataG4 = testData['G4'].values
testDataPartition = testData['Partition'].values

print("---loading test data done.---")


---loading test data done.---


In [ ]:
testDataNumbers = [100000, 10000, 1000, 1579746]
testDataPartition = [810, 127, 28, 14932]

In [ ]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []

for num in testDataNumbers:
    #Euler's Totient Function Partitions fist 50
    feature = []
    feature = compute_coprimes(num)
    mergedTestNumber.append(feature)

for num in testDataPartition:
    mergedTestLabel.append(num)

In [ ]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [ ]:
evaluation = model.evaluate(mergedArray, mergedLabel)
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)



2/2 [==============================] - 1s 45ms/step - loss: 1433156480.0000 - accuracy: 0.0000e+00
Loss: 1433156480.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 9ms/step


In [ ]:
"""
順便計算了G4
本模型的訓練目的是建立原始分區與 G4 之間的關係，
因此標籤資料需要除以 G4。下方的測試資料中，數值 7984914 的正確答案是 58853，而模型預測值為 1.721389。

實際上，正確答案應該是 1.721389 乘以 G4，這樣才能得到模型的最終正確答案。
"""
# cal G4
def G4(n) -> int:
  return int((n / np.log((n/2))**2)+0.5)

testDataG4 = [G4(i) for i in testDataNumbers]


print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataPartition[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")

測試五十筆資料
第0筆資料為數字:	7984914	|正確答案為:58853	模型預估為[1.721389], 四捨五入到[2.]
第1筆資料為數字:	7050792	|正確答案為:51667	模型預估為[1.6746786], 四捨五入到[2.]
第2筆資料為數字:	7852768	|正確答案為:31367	模型預估為[0.9302076], 四捨五入到[1.]
第3筆資料為數字:	7807448	|正確答案為:26161	模型預估為[0.7781038], 四捨五入到[1.]
第4筆資料為數字:	7695230	|正確答案為:31001	模型預估為[0.9379368], 四捨五入到[1.]
第5筆資料為數字:	7109000	|正確答案為:28859	模型預估為[0.9379368], 四捨五入到[1.]
第6筆資料為數字:	7268430	|正確答案為:64326	模型預估為[2.0327833], 四捨五入到[2.]
第7筆資料為數字:	7147384	|正確答案為:22306	模型預估為[0.71668863], 四捨五入到[1.]
第8筆資料為數字:	7555772	|正確答案為:27371	模型預估為[0.8386079], 四捨五入到[1.]
第9筆資料為數字:	7066874	|正確答案為:21654	模型預估為[0.7017362], 四捨五入到[1.]
第10筆資料為數字:	7632520	|正確答案為:36879	模型預估為[1.1266344], 四捨五入到[1.]
第11筆資料為數字:	7039812	|正確答案為:46596	模型預估為[1.5313977], 四捨五入到[2.]
第12筆資料為數字:	7670862	|正確答案為:46176	模型預估為[1.4010736], 四捨五入到[1.]
第13筆資料為數字:	7402232	|正確答案為:22333	模型預估為[0.7017362], 四捨五入到[1.]
第14筆資料為數字:	7432508	|正確答案為:22866	模型預估為[0.7137461], 四捨五入到[1.]
第15筆資料為數字:	7230862	|正確答案為:22140	模型預估為[0.7017362], 四捨五入到[1.]
第16筆資料為數字:	7305346	|正確答案為:22305	模型預估為[0.70

# +N 才要執行正規

In [ ]:
"""
+n 調整
+n 代表對原本的partition進行一些修改。
在這裡，+n 是將partition針對 G4 進行正規化，也就是將標籤 (label) 除以 G4。因此，我們需要在這一段將標籤正規化回來。

這就是為什麼在計算中包含了乘以 testDataG4 的部分。
"""
print (f"將50筆測資正規化回G4")
normalizedTest = []
normalizedPred = []
for index in range(len(testDataLabel)):
  normalizedTest.append(testDataLabel[index]*testDataG4[index]) # normalize ground truth
  normalizedPred.append(predictions[index]*testDataG4[index])  # normalize predition



將50筆測資正規化回G4


In [ ]:
"""
正規化後的測試
"""
print (f"將50筆測資正規化回G4")
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")

將50筆測資正規化回G4
第0筆資料為數字:	7984914	|正確答案 = 58853	模型預估 = 59492
第1筆資料為數字:	7050792	|正確答案 = 51667	模型預估 = 51955
第2筆資料為數字:	7852768	|正確答案 = 31367	模型預估 = 31686
第3筆資料為數字:	7807448	|正確答案 = 26161	模型預估 = 26372
第4筆資料為數字:	7695230	|正確答案 = 31001	模型預估 = 31392
第5筆資料為數字:	7109000	|正確答案 = 28859	模型預估 = 29306
第6筆資料為數字:	7268430	|正確答案 = 64326	模型預估 = 64750
第7筆資料為數字:	7147384	|正確答案 = 22306	模型預估 = 22498
第8筆資料為數字:	7555772	|正確答案 = 27371	模型預估 = 27626
第9筆資料為數字:	7066874	|正確答案 = 21654	模型預估 = 21813
第10筆資料為數字:	7632520	|正確答案 = 36879	模型預估 = 37441
第11筆資料為數字:	7039812	|正確答案 = 46596	模型預估 = 47445
第12筆資料為數字:	7670862	|正確答案 = 46176	模型預估 = 46765
第13筆資料為數字:	7402232	|正確答案 = 22333	模型預估 = 22708
第14筆資料為數字:	7432508	|正確答案 = 22866	模型預估 = 23179
第15筆資料為數字:	7230862	|正確答案 = 22140	模型預估 = 22252
第16筆資料為數字:	7305346	|正確答案 = 22305	模型預估 = 22450
第17筆資料為數字:	7349124	|正確答案 = 47254	模型預估 = 47959
第18筆資料為數字:	7134356	|正確答案 = 22965	模型預估 = 23475
第19筆資料為數字:	7990454	|正確答案 = 23942	模型預估 = 24267
第20筆資料為數字:	7024954	|正確答案 = 22002	模型預估 = 21941
第21筆資料為數字:	7715734	|正確答案 = 2551

<ipython-input-14-7ef344336dce>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")


In [ ]:
"""
顯示尚未正規化前的資料
"""
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	1.702867347  		/  	[2.] = [0.8514337] 
1	1.665401625  		/  	[2.] = [0.8327008] 
2	0.920828024  		/  	[1.] = [0.92082804] 
3	0.771866828  		/  	[1.] = [0.7718668] 
4	0.926237389  		/  	[1.] = [0.9262374] 
5	0.923612865  		/  	[1.] = [0.9236129] 
6	2.019479073  		/  	[2.] = [1.0097395] 
7	0.71056174  		/  	[1.] = [0.71056175] 
8	0.830867449  		/  	[1.] = [0.83086747] 
9	0.696603547  		/  	[1.] = [0.69660354] 
10	1.109712659  		/  	[1.] = [1.1097126] 
11	1.503977776  		/  	[2.] = [0.7519889] 
12	1.383434641  		/  	[1.] = [1.3834347] 
13	0.690122259  		/  	[1.] = [0.69012225] 
14	0.704094356  		/  	[1.] = [0.70409435] 
15	0.698204988  		/  	[1.] = [0.698205] 
16	0.697181913  		/  	[1.] = [0.69718194] 
17	1.469369561  		/  	[1.] = [1.4693695] 
18	0.732712976  		/  	[1.] = [0.732713] 
19	0.692326664  		/  	[1.] = [0.69232666] 
20	0.711460457  		/  	[1.] = [0.7114605] 
21	0.760450346  		/  	[1.] = [0.76045036] 
22	1.665650216  		/  	[2.] = [0.8328251] 
23	2.019310047  		/

In [ ]:
"""
顯示正確答案與預測答案的差距量
"""
D = []
for index in range (len(testDataPartition)):
  D.append(testDataPartition[index] - int(normalizedPred[index]))


print ("\t正確答案  "  , "\t-" ,"  \t模型預測\t\t差距量")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataPartition[index]}\t\t\t{int(normalizedPred[index])}\t=\t{abs(D[index])}")

	正確答案   	-   	模型預測		差距量
0	58853			59492	=	639
1	51667			51955	=	288
2	31367			31686	=	319
3	26161			26372	=	211
4	31001			31392	=	391
5	28859			29306	=	447
6	64326			64750	=	424
7	22306			22498	=	192
8	27371			27626	=	255
9	21654			21813	=	159
10	36879			37441	=	562
11	46596			47445	=	849
12	46176			46765	=	589
13	22333			22708	=	375
14	22866			23179	=	313
15	22140			22252	=	112
16	22305			22450	=	145
17	47254			47959	=	705
18	22965			23475	=	510
19	23942			24267	=	325
20	22002			21941	=	61
21	25511			25808	=	297
22	53823			54115	=	292
23	65325			65866	=	541
24	23978			24215	=	237
25	45171			45715	=	544
26	22227			22411	=	184
27	46283			46616	=	333
28	23166			23260	=	94
29	22558			22854	=	296
30	56207			56614	=	407
31	28538			29093	=	555
32	30025			30708	=	683
33	21786			22079	=	293
34	33860			34467	=	607
35	46031			46267	=	236
36	32376			32361	=	15
37	25863			26294	=	431
38	43561			44140	=	579
39	21553			21734	=	181
40	21393			21634	=	241
41	44362			44719	=	357
42	51956			52256	=	300


<ipython-input-17-1021a7c2349b>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  D.append(testDataPartition[index] - int(normalizedPred[index]))
<ipython-input-17-1021a7c2349b>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"{index}\t{testDataPartition[index]}\t\t\t{int(normalizedPred[index])}\t=\t{abs(D[index])}")


## 把所有資料zip

In [ ]:
"""
顯示正確答案與預測答案的差距量
"""
gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]


normalizedPred_int = [int(arr) for arr in normalizedPred]
combined_data = list(zip(testDataNumbers, testDataPartition, normalizedPred_int, gap))
sort_data = sorted(combined_data, key = lambda x : x[0])


print (f"test on {testDataPath[39::]}")
print (f"測試數字/正確答案/預測答案/差距量\n")
for i in sort_data:
  print (i)

test on test_set_G4_7M.csv
測試數字/正確答案/預測答案/差距量

(7000034, 21393, 21634, 241)
(7024954, 22002, 21941, 60)
(7037344, 21553, 21734, 181)
(7039812, 46596, 47445, 849)
(7049660, 28538, 29093, 555)
(7050792, 51667, 51955, 288)
(7066874, 21654, 21813, 159)
(7097916, 51956, 52256, 300)
(7109000, 28859, 29306, 447)
(7121414, 22558, 22854, 296)
(7134356, 22965, 23475, 510)
(7147384, 22306, 22498, 192)
(7166374, 21786, 22079, 293)
(7177008, 43561, 44140, 579)
(7229336, 22227, 22411, 184)
(7230862, 22140, 22252, 112)
(7236930, 59226, 59533, 307)
(7243074, 44362, 44719, 357)
(7259704, 23166, 23260, 94)
(7268224, 22013, 22351, 338)
(7268430, 64326, 64750, 424)
(7305346, 22305, 22450, 145)
(7349124, 47254, 47959, 705)
(7389732, 53823, 54115, 292)
(7399392, 65325, 65866, 541)
(7402232, 22333, 22708, 375)
(7432508, 22866, 23179, 313)
(7472958, 45171, 45715, 544)
(7502200, 30025, 30708, 683)
(7555772, 27371, 27626, 255)
(7576878, 46031, 46267, 236)
(7606482, 50630, 51152, 522)
(7632520, 36879, 37441, 562

<ipython-input-20-048f8f5163d8>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]
<ipython-input-20-048f8f5163d8>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  normalizedPred_int = [int(arr) for arr in normalizedPred]


## RMSE and error rate

In [ ]:
"""
計算三參數
MSE RMSE 跟 error rate
RMSE = sqrt(MSE)
error = RMSE / mean of partition
"""
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse



mean = testData['Partition'].mean()

#LN MSE
MSE = int(calculate_mse(normalizedPred, testDataPartition))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE} / ({MSE/1000000})\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

MSE : 163631 / (0.163631)
RMSE : 404.5132877916373
Error Rate : 0.01148917831049691  ->  1.14892%


<ipython-input-24-2d4cd73154ce>:29: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  MSE = int(calculate_mse(normalizedPred, testDataPartition))


In [ ]:
import math

def prime_factors(n):
      i = 2
      factors = []
      while i * i <= n:
          if n % i:
              i += 1
          else:
              n //= i
              factors.append(i)
      if n > 1:
          factors.append(n)
      return factors


def goldbach_approx(n):
    C2 = 0.66016  # 孪生素数常数
    product = 1
    for p in prime_factors(n):
        if p == 2:
            continue
        product *= (p - 1) / (p - 2)
    return 2 * C2 * n / (math.log(n))**2 * product

n = 100
result = goldbach_approx(n)
print(result)


11.067902035483288


<ipython-input-23-c8dcf57967f2>:25: RuntimeWarning: invalid value encountered in log
  return x / np.log(x) - y
<ipython-input-23-c8dcf57967f2>:27: RuntimeWarning: invalid value encountered in log
  return (np.log(x) - 1) / np.log(x)**2


ValueError: 迭代次数超过限制